<a href="https://colab.research.google.com/github/victorianieto/iniciacion-git-y-github/blob/master/01_basic_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CARTOframes basic usage

## What do I need to start using CARTOframes?

If you don't have yet an account at CARTO, and you want to start learning how to use CARTOframes, you only need the `cartoframes` library. Take a look to the **Sources** examples to know how to visualize data from a DataFrame or a GeoJSON.

If you already have an account, you can start analyzing and visualizing your data!

This notebook summarizes all the steps you need to understand the basics of CARTOframes visualization. Each section is explained in more detail in our [Guides](https://carto.com/developers/cartoframes/guides/) and in the rest of the examples.

## Choose your Data

In this quickstart guide, we'll be using the [`spend_data`](https://cartovl.carto.com/tables/spend_data/public/map) Dataset, which contains customer activity information in the city of Barcelona.

## Credentials setup

Authenticate with CARTO.

In [3]:
%pip install cartoframes==1.0b3

     |████████████████████████████████| 194kB 2.6MB/s 
     |████████████████████████████████| 163kB 36.1MB/s 
     |████████████████████████████████| 245kB 42.9MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for pyrestcli: filename=pyrestcli-0.6.11-cp36-none-any.whl size=8500 sha256=de400291011da5d6c9d8f2afb01c42f5ae9cde79880bbb069331011e118132c8
  Stored in directory: /root/.cache/pip/wheels/2f/bb/11/396a62e2d1e718f2bfb02b66726240fbc8d98640bfc0cf1688
Successfully built pyrestcli
  Found existing installation: google-cloud-bigquery 1.14.1
    Uninstalling google-cloud-bigquery-1.14.1:
      Successfully uninstalled google-cloud-bigquery-1.14.1
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
## DATOS: TRANSACCIONES DE CREDITO EN BARCELONA

In [5]:
from cartoframes.auth import set_default_credentials
from cartoframes.viz import Map, Layer

set_default_credentials('cartovl') ##nuestro usuario

Map(Layer('spend_data'))

## Change the viewport, bounds and basemap

In [8]:
from cartoframes.viz import Map, Layer, basemaps

Map(
    Layer('spend_data'),
    viewport={'zoom': 12.03, 'lat': 41.4, 'lng': 2.19}, ## zoom: 1 es todo el mapa
    basemap=basemaps.darkmatter,
    show_info=True
)

## Get your data using a SQL query

In [3]:
from cartoframes.viz import Map, Layer

Map(
    Layer('SELECT * FROM spend_data WHERE amount > 150 AND amount < 200')
)

## Modify and upload data

Here your need to use your credentials in order to create a new dataset in your CARTO account

In [11]:
from cartoframes.auth import Credentials
from cartoframes.data import Dataset

# here your are still using the credentials from cartovl account
dataset_from_cartovl = Dataset('SELECT * FROM spend_data WHERE amount > 150 AND amount < 200')
df = dataset_from_cartovl.download()

# create a Credentials object with your account parameters to use it in the upload method
credentials = Credentials(
    username='victoria-nieto',
    api_key='c37a5a52ba9326bd1116862682a574123f2b0c66' #API Key de Carto
)

dataset_for_your_account = Dataset(df)
# HAGO UN UPLOAD EN MI CUENTA Y DEBERIA TENER SPEND_DATA ALLI
dataset_for_your_account.upload(table_name='spend_data', if_exists='replace', credentials=credentials)

## Change the style

In [15]:
Map(
    Layer(
        'spend_data',
        'color: ramp($category, sunset)'
    ),
    viewport={'zoom': 12.03, 'lat': 41.4, 'lng': 2.19}
)

## Add legends

In [16]:
Map(
    Layer(
        'spend_data',
        'color: ramp($category, bold)',
        legend={
            'type': 'color-bins',
            'title': 'Categories'
        }
    ),
    viewport={'zoom': 12.03, 'lat': 41.4, 'lng': 2.19}
)

## Add popups

In [17]:
Map(
    Layer(
        'spend_data',
        'color: ramp($category, bold)',
        popup={
            'hover': [{
                'title': 'Category',
                'value': '$category'
            }, {
                'title': 'Hour',
                'value': '$hour'
            }]
        }
    ),
    viewport={'zoom': 12.03, 'lat': 41.4, 'lng': 2.19}
)

## Use built in helpers

In [18]:
from cartoframes.viz.helpers import color_bins_layer

Map(
    color_bins_layer('spend_data','amount', 'Spent Amount €'), # columna amount
    viewport={'zoom': 12.03, 'lat': 41.4, 'lng': 2.19}
)

## Publish and share your visualizations

In [19]:
visualization = Map(
    color_bins_layer('spend_data','amount', 'Spent Amount €'),
        viewport={'zoom': 12.03, 'lat': 41.4, 'lng': 2.19}

)

kuviz = visualization.publish('spent_amount_in_barcelona') # PUBLICAR EL MAPA

print(kuviz['url'])

https://cartovl.carto.com/kuviz/ab22c1e6-5e5e-4cc7-8c8d-e60b65a4b83c
